<a href="https://colab.research.google.com/github/mleyvaz/IO/blob/main/Copia_de_vrp_capacity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# vrp_capacity

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/master/examples/notebook/constraint_solver/vrp_capacity.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/master/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/master/ortools/constraint_solver/samples/vrp_capacity.py"><img src="https://raw.githubusercontent.com/google/or-tools/master/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [1]:
!pip install ortools

     |████████████████████████████████| 14.0MB 341kB/s 
     |████████████████████████████████| 133kB 46.8MB/s 
     |████████████████████████████████| 1.0MB 39.2MB/s 
ERROR: tensorflow-metadata 0.27.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.
  Found existing installation: absl-py 0.10.0
    Uninstalling absl-py-0.10.0:
      Successfully uninstalled absl-py-0.10.0
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [6]:
# Copyright 2010-2018 Google LLC
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# [START program]
"""Capacited Vehicles Routing Problem (CVRP)."""

# [START import]
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]


# [START data_model]
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [[   0.        ,   58.96033328,  423.83701042,  802.77455621,
         832.92540752,  925.27301166,  988.86855674, 1107.27264687,
        1143.04199017, 1233.90085185, 1290.46535858],
       [  58.96033328,    0.        ,  472.92246944,  857.63279438,
         888.36268089,  981.4189883 , 1044.41726613, 1161.87259956,
        1197.43477234, 1287.86705234, 1344.13354768],
       [ 423.83701042,  472.92246944,    0.        ,  403.09599167,
         438.25146263,  536.2846193 ,  591.85371715,  700.93923538,
         735.06136676,  822.90488709,  877.71991985],
       [ 802.77455621,  857.63279438,  403.09599167,    0.        ,
          38.43921435,  136.03615769,  188.95288355,  304.60275836,
         340.68414228,  432.58077639,  490.11432136],
       [ 832.92540752,  888.36268089,  438.25146263,   38.43921435,
           0.        ,   98.64194747,  156.09537149,  276.7166493 ,
         313.45218008,  406.31496908,  464.44714705],
       [ 925.27301166,  981.4189883 ,  536.2846193 ,  136.03615769,
          98.64194747,    0.        ,   70.53798126,  198.02823082,
         235.12725448,  328.10514245,  386.71679689],
       [ 988.86855674, 1044.41726613,  591.85371715,  188.95288355,
         156.09537149,   70.53798126,    0.        ,  127.50840364,
         164.6394248 ,  257.73660605,  316.3627339 ],
       [1107.27264687, 1161.87259956,  700.93923538,  304.60275836,
         276.7166493 ,  198.02823082,  127.50840364,    0.        ,
          37.21067051,  130.54269378,  189.13835888],
       [1143.04199017, 1197.43477234,  735.06136676,  340.68414228,
         313.45218008,  235.12725448,  164.6394248 ,   37.21067051,
           0.        ,   93.34749327,  151.93102119],
       [1233.90085185, 1287.86705234,  822.90488709,  432.58077639,
         406.31496908,  328.10514245,  257.73660605,  130.54269378,
          93.34749327,    0.        ,   58.62798734],
       [1290.46535858, 1344.13354768,  877.71991985,  490.11432136,
         464.44714705,  386.71679689,  316.3627339 ,  189.13835888,
         151.93102119,   58.62798734,    0.        ]]
    # [START demands_capacities]
    data['demands'] = [15, 0.3, 0.8, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 15]
    data['vehicle_capacities'] = [15, 15]
    # [END demands_capacities]
    data['num_vehicles'] = 2
    data['depot'] = 1
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    # [END solution_printer]


"""Solve the CVRP problem."""
# Instantiate the data problem.
# [START data]
data = create_data_model()
# [END data]

# Create the routing index manager.
# [START index_manager]
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
# [END index_manager]

# Create Routing Model.
# [START routing_model]
routing = pywrapcp.RoutingModel(manager)

# [END routing_model]

# Create and register a transit callback.
# [START transit_callback]
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
# [END transit_callback]

# Define cost of each arc.
# [START arc_cost]
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# [END arc_cost]

# Add Capacity constraint.
# [START capacity_constraint]
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')
# [END capacity_constraint]

# Setting first solution heuristic.
# [START parameters]
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(1)
# [END parameters]

# Solve the problem.
# [START solve]
solution = routing.SolveWithParameters(search_parameters)
# [END solve]

# Print solution on console.
# [START print_solution]
if solution:
    print_solution(data, manager, routing, solution)
# [END print_solution]



Route for vehicle 0:
 1 Load(0.3) ->  3 Load(0.6) ->  4 Load(0.8999999999999999) ->  5 Load(1.2) ->  6 Load(1.5) ->  7 Load(1.8) ->  8 Load(2.1) ->  9 Load(2.4) ->  10 Load(17.4) ->  2 Load(18.2) ->  1 Load(18.2)
Distance of the route: 2727m
Load of the route: 18.2

Route for vehicle 1:
 1 Load(0.3) ->  0 Load(15.3) ->  1 Load(15.3)
Distance of the route: 116m
Load of the route: 15.3

Total distance of all routes: 2843m
Total load of all routes: 33.5


In [ ]:
10000*df.values


array([[   0.        ,   58.96033328,  423.83701042,  802.77455621,
         832.92540752,  925.27301166,  988.86855674, 1107.27264687,
        1143.04199017, 1233.90085185, 1290.46535858],
       [  58.96033328,    0.        ,  472.92246944,  857.63279438,
         888.36268089,  981.4189883 , 1044.41726613, 1161.87259956,
        1197.43477234, 1287.86705234, 1344.13354768],
       [ 423.83701042,  472.92246944,    0.        ,  403.09599167,
         438.25146263,  536.2846193 ,  591.85371715,  700.93923538,
         735.06136676,  822.90488709,  877.71991985],
       [ 802.77455621,  857.63279438,  403.09599167,    0.        ,
          38.43921435,  136.03615769,  188.95288355,  304.60275836,
         340.68414228,  432.58077639,  490.11432136],
       [ 832.92540752,  888.36268089,  438.25146263,   38.43921435,
           0.        ,   98.64194747,  156.09537149,  276.7166493 ,
         313.45218008,  406.31496908,  464.44714705],
       [ 925.27301166,  981.4189883 ,  536.2846193